# Word Sense Disambiguation using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 30 different words. 

In [1]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
import torch.optim as optim

# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:0')
#device = torch.device('cpu')



# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 30 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [ ]:
import random


def data_split(path_to_dataset):

    with open (path_to_dataset, "r") as f:
        content = f.readlines()
    random.shuffle(content)                   # shuffle sentences to select randomly for train/test set
    samples = len(content)
    train = (samples*80)//100
    train_set = content[:train]
    test_set = content[train:]


    with open('train.csv', 'w') as output:
        for data in train_set:
            output.write(data)

    with open('test.csv', 'w') as output:
        for data in test_set:
            output.write(data)

path = "./wsd-data/wsd_data.txt"
data_split(path)

### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [1]:
import collections
from collections import Counter

def mcs_baseline(data):
    form_sense = {}
    mcs = {}
    for_accuracy = {}

    with open (data, "r") as f:
        content = f.readlines()
    for data in content:
        new = data.strip().split('\t')
        if new[1] not in form_sense:
            form_sense[(new[1])] = []
        form_sense[(new[1])].append(new[0])
    for key, value in form_sense.items():
        counter=collections.Counter(value)
        counter = dict(Counter(counter))
        form_sense[key] = counter
    for key1, dic in form_sense.items(): # loop again due to run time error
        total = sum(dic.values()) # sum the number of senses for each lemma
        sorted_dic = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1])} # sort dictionary
        sort_values = [sorted_dic[x] for x in list(sorted_dic)] # get sorted values as list
        mcs[key1] = [float("{:.2}".format(sort_values[-1]/total)),len(sort_values)]
        for_accuracy[(list(dic)[-1])] = float("{:.2}".format(sort_values[-1]/total))
        #key of the for_accuracy dictionary is the most frequent sense for a specific lemma
        #value is the mcs accuracy for the corresponding lemma

    total_accuracy = sum(list(for_accuracy.values()))/len(mcs)
    print("\t\tAccuracy\t Number of senses")
    [print(key,'  \t',int(value[0]*100),"%  \t\t",value[1]) for key, value in mcs.items()]
    print("\nBaseline accuracy: ", str(int(total_accuracy*100))+"%")


path = "./wsd-data/wsd_data.txt"
mcs_baseline(path)

		Accuracy	 Number of senses
keep.v   	 39 %  		 11
national.a   	 20 %  		 6
build.v   	 21 %  		 10
place.n   	 24 %  		 7
position.n   	 20 %  		 6
serve.v   	 16 %  		 9
hold.v   	 15 %  		 11
line.n   	 85 %  		 11
see.v   	 63 %  		 11
time.n   	 28 %  		 5
physical.a   	 24 %  		 6
follow.v   	 15 %  		 11
regular.a   	 22 %  		 8
bad.a   	 61 %  		 4
force.n   	 16 %  		 8
professional.a   	 22 %  		 5
security.n   	 20 %  		 7
positive.a   	 35 %  		 5
point.n   	 36 %  		 8
common.a   	 25 %  		 4
find.v   	 23 %  		 10
life.n   	 22 %  		 9
order.n   	 22 %  		 5
bring.v   	 21 %  		 8
active.a   	 32 %  		 5
extend.v   	 18 %  		 7
case.n   	 20 %  		 8
lead.v   	 18 %  		 8
critical.a   	 27 %  		 5
major.a   	 30 %  		 4

Baseline accuracy:  28%


### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()* 

In [3]:

def dataloader():
    ddir = './'
    whitespacer = lambda x: x.split(' ')

    # "fields" that process the different columns in our CSV files
    # similar to assignment 3 given example
    
    WORD_SENSE = Field(tokenize    = whitespacer,
                       batch_first = True)
    
    POSITION = Field(batch_first = True,
                     sequential = False,
                     use_vocab = False)
    
    LABELS = Field(tokenize    = whitespacer,
                   batch_first = True) # enforce the (batch, words) structure
    
    CONTEXT = Field(tokenize    = whitespacer,
                    lower       = True,
                    batch_first = True)

    # read the csv files
    train, test = TabularDataset.splits(path   = ddir,
                                        train  = 'train.csv',
                                        test   = 'test.csv',
                                        format = 'csv',
                                        fields = [('word_sense', WORD_SENSE),
                                                  ('lemma', LABELS),
                                                  ('position', POSITION),
                                                  ('context', CONTEXT)],
                                        skip_header = False,
                                        csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'½'})

    # build vocabularies based on what our csv files contained and create word2id mapping
    CONTEXT.build_vocab(train)
    LABELS.build_vocab(train)
    WORD_SENSE.build_vocab(train)

    # create batches from our data, and shuffle them for each epoch
    train_iter, test_iter = BucketIterator.splits((train, test),
                                                  batch_size        = 8,
                                                  sort_within_batch = True,
                                                  sort_key          = lambda x: len(x.context),
                                                  shuffle           = True,
                                                  device            = device)

    return train_iter, test_iter, WORD_SENSE, POSITION, LABELS, CONTEXT 
dataloader()

(<torchtext.legacy.data.iterator.BucketIterator at 0x7f53fc2fa410>,
 <torchtext.legacy.data.field.Field at 0x7f541b50c610>)

# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


In [4]:
# documentation consulted https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

class WSDModel_approach1(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocabulary_dim, sense_num): 
        super(WSDModel_approach1, self).__init__()
        self.embeddings = nn.Embedding(vocabulary_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, bidirectional=True)
        self.classifier = nn.Linear(hidden_dim*2, sense_num) 
    
    def forward(self, batch):
        # embedd the words in out sentence
        embedded_batch = self.embeddings(batch.context)
        # create contextualized representations for our words

        contextualized_representations, _ = self.rnn(embedded_batch)
        # 1) output features of the hidden state from all time steps
        # 2) hidden state for the step 'n' which we do not need in this case 
        
        out = contextualized_representations[range(contextualized_representations.shape[0]),batch.position]
        predictions = self.classifier(out) 
        
        return predictions


In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [5]:
class WSDModel_approach2(nn.Module):
    def __init__(self,embedding_dim, hidden_dim, vocabulary_dim, sense_num):
        super(WSDModel_approach2, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocabulary_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, bidirectional = True)
        self.classifier = nn.Linear(hidden_dim*2, sense_num) 
        
    
    def forward(self, batch):
        # embedd the words in out sentence
        embedded_batch = self.embeddings(batch.context)
        
        contextualized_representations, (h0, c0) = self.rnn(embedded_batch)
        #the new shape of our output is: batch size, sequence length, hidden size
        # decode the hidden state only for the last time step

        forward = h0[0,:,:]
        backward = h0[1,:,:]
        conc = torch.cat((forward,backward),1)

        # h_n: tensor with hidden state
        # c_n: tensor with cell state

        predictions = self.classifier(conc)
        
        return predictions
    

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [6]:
# you can change these numbers to suit your needs :)
hyperparameters = {'epochs':3,
                   'batch_size':8,
                   'learning_rate':0.001,
                   'embedding_dim':32,
                   'hidden_dim':32}

In [7]:
# documentation consulted https://pytorch.org/docs/stable/optim.html


train_iter, test_iter, sense, position, labels, vocab = dataloader()

WSDModel1 = WSDModel_approach1(hyperparameters['embedding_dim'], 
                               hyperparameters['hidden_dim'],
                               len(vocab.vocab),
                               len(sense.vocab))

WSDModel2 = WSDModel_approach2(hyperparameters['embedding_dim'], 
                               hyperparameters['hidden_dim'],
                               len(vocab.vocab),
                               len(sense.vocab))

loss_function = nn.CrossEntropyLoss() # the only Loss function used so far, so I follow the map (also, best for classification problems?)
WSDModel1.to(device)
WSDModel2.to(device)
optimizer1 = optim.Adam(WSDModel1.parameters(), hyperparameters ['learning_rate'])
optimizer2 = optim.Adam(WSDModel2.parameters(), hyperparameters ['learning_rate'])



In [8]:
#TRAIN MODEL 

import numpy as np

def train_model(model,name):
    model.train()
    total_loss = 0
    for epoch in range(hyperparameters['epochs']):
        for i, batch in enumerate(train_iter):
            
            senses    = batch.word_sense

            # run sentences through the model
            output = model(batch) #forward parameter: batch


            batch_loss = loss_function(output, senses.reshape(-1))
            total_loss += batch_loss.item()
            
            # report results
            print(epoch, (i+1)*senses.size(0), np.round(total_loss/(i+1),4),end='\r')

            # calculate gradients
            batch_loss.backward()
            # update model weights
            optimizer1.step()
            # reset gradients
            optimizer1.zero_grad()
        print()
    torch.save(model, "./"+name+ ".pt")       
    

train_model(WSDModel1,"WSDModel1")
train_model(WSDModel2,"WSDModel2")
# test model after all epochs are completed

0 60840 1.5809
1 60840 2.5694
2 60840 3.4174
0 60840 5.4067
1 60840 10.8134
2 60840 16.22


In [9]:

# TEST MODEL

def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())
    

def test_model(model,name):
    test_loss = 0
    model.eval()
    running_accuracy = 0.00\
    num_senses = len(sense.vocab)
    
    for i, batch in enumerate(test_iter):
        senses    = batch.word_sense
        classes   = batch.lemma
        
        with torch.no_grad(): # dont collect gradients when testing
            output = model(batch)
        batch_loss = loss_function(output, senses.reshape(-1))
        test_loss += batch_loss.item()
        running_accuracy += accuracy(output, senses.reshape(-1))

    
    running_accuracy /= len(test_iter) # a tensor that contains accuracy and device
    print(name + " accuracy:",np.round(running_accuracy.item(),2)) # print accuracy only

WSDModel1 = torch.load("./WSDModel1.pt")
WSDModel2 = torch.load("./WSDModel2.pt")

test_model(WSDModel1,"WSDModel1")
test_model(WSDModel2,"WSDModel2")


WSDModel1 accuracy: 0.67
WSDModel2 accuracy: 0.01


### This is completely weird. The accuracy for model1 looks fine, but we have no idea why we get this result for model2, although everything seem to be correct.
### Also, we tried multiple ways but did not manage to compute the accuracy for each label :/

# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the BertTokenizer to transform the sentence into integers. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either the the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [13]:
import random
from transformers import BertTokenizer, BertModel, BertForMaskedLM


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def dataloader_for_bert(path_to_file, batch_size):
    #numerical representations of tokens building the sequences that will be used as input by the model
    token_indices = [] 
    #The attention mask is an optional argument used when batching sequences together. This argument indicates to the model which tokens should be attended to, and which should not.
    attention_masks = []
    labels = []
    with open (path_to_file, "r") as f:
        content = f.readlines()    

    random.shuffle(content)

    for i in range((len(content)//batch_size)+1):
        if len(content) < batch_size:
            batches = content
            del content[:]
        else:
            batches = content[:batch_size]
            del content[:batch_size]
        for batch in batches:
            context = batch.split("\t")[3]
            sense = batch.split("\t")[0]
            labels.append(sense)
            tokenized = tokenizer.batch_encode_plus(context, padding=True, return_attention_mask=True).to(device)
            #The tokenizer returns a dictionary with all the arguments necessary for its corresponding model to work properly. The token indices are under the key “input_ids”:
            token_indices.append(tokenized['input_ids'])
            attention_masks.append(tokenized['attention_mask'])
            
    return labels, token_indices, attention_masks



In [14]:
labels2, token_indices2, attention_masks2 = dataloader_for_bert("/content/drive/MyDrive/04-word-sense-diasambiguation/test.csv",4)
labels1, token_indices1, attention_masks1 = dataloader_for_bert("/content/drive/MyDrive/04-word-sense-diasambiguation/train.csv",4)
test_iter = zip(labels2,token_indices2,attention_masks2)
train_iter = zip(labels1,token_indices1,attention_masks1)


AttributeError: ignored

In [37]:
class BERT_WSD(nn.Module):
    def __init__(self, in_put):
        super(BERT_WSD, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, in_put)
    
    def forward(self, batch): # batch contains input_ids, masks and sense labels
        try:
            out = self.bert(batch.token_indices1,batch.attention_masks1)
        except:
            out = self.bert(batch.token_indices2,batch.attention_masks2)
        hidden_repr = out[:,0,:]
        predictions = self.classifier(hidden_repr)
        return predictions

In [38]:
# Documentation
# https://mccormickml.com/2019/07/22/BERT-fine-tuning/
# https://medium.com/analytics-vidhya/a-gentle-introduction-to-implementing-bert-using-hugging-face-35eb480cff3

from transformers import AdamW, get_linear_schedule_with_warmup

loss_function = nn.CrossEntropyLoss()
model = BERT_WSD(len(labels.vocab))
model.to(device)
optimizer = optim.AdamW(model.parameters(), hyperparameters ['learning_rate'])

# Not sure if we need the scheduler, but we found it in 2 tutorials
total_steps = hyperparameters ['batch_size'] * hyperparameters ['epochs']
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 1000, 
                                            num_training_steps = total_steps)

# TRAINING
total_loss = 0
model.train()
for _ in range(hyperparameters ['epochs']):
    for i, batch in enumerate(train_iter):
        output = model(batch)

    # Converting to cuda tensors
        batch_var = (t.to(device) for t in batch)
        labels, indices, attn_masks = batch_var

        batch_loss = loss(output, labels)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update the learning rate.
        scheduler.step()
    print()

# TEST   
def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())    
# test model after all epochs are completed
test_loss = 0
running_accuracy = 0.00
model.eval()
for i, batch in enumerate(test_iter):
    batch_var = (t.to(device) for t in batch)
    labels, indices, attn_masks = batch_var
    with torch.no_grad():
        output = model(batch)
    batch_loss = loss(output, labels)
    test_loss += batch_loss.item()
    running_accuracy += accuracy(output, labels)
running_accuracy /= len(test_iter) # a tensor that contains accuracy and device
print("Model accuracy:",np.round(running_accuracy.item(),2)) # print accuracy only
  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AttributeError: ignored

# 3. Evaluation

Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

In the first approach we try to predict the sense of a specific word based on the representation of the target word (using the index). In the second approach we consider only the final hidden state, which we believe it means that all the words(context/sentence) are used to predict the sense of a word (since we provide no specific index to find the sense of the target word).

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

Unfortunately, we did not manage to calculate the per-word-form accuracy and also train the BERT model. We will attempt though to provide some explanations according to our intuition.

We suspect that Model1 with 67% accuracy would be definetely better than the baseline, but we cannot be sure concerning the performance of Model2. 

We would also suspect that maybe words with fewer senses are easier to predict because there are less chances for our model to be wrong when trying to predict between less classes. At the same time though this is a matter of probability so it is not a stable conclusion. Thus, depending on our data, eventually the number of senses for each lemma does not matter after all.

How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

We are pretty confident that BERT should present the best results out of all our models, especially since pre-trained embeddings are applied. BERT takes into consideration the meaning of the whole sentence (at least this is what we tried to do in the forward function), so it could be compared to Model's2 approach. 

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

The very first suggestion we always propose to improve our models' perfomances is to use more data. But as we know, more data is not always the best approach, if data is not selected wisely. For instance, if we add more data but concern only specidic senses, then it won't make a great difference. A wider variety in the senses category is needed for our models to perform better. Also, we should test different epochs, number of batches, embedding dimensionality, hidden layers and learning rate.

For BERT, a "clever" optimization can lead to improvement (learning rate,number of warmup steps, training steps, larger batch size). Batch size was suggested to be small, so a greater size could bring better results.

# Readings:

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf